In [146]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F

class TwoPathConv(nn.Module):
    def __init__(self):
        super(TwoPathConv, self).__init__()
        self.upper_layer1 = nn.Sequential(
            nn.Conv2d(4,64,7),
            nn.ReLU(),
            nn.MaxPool2d((4,4),stride = 1)
        )
        self.upper_layer2 = nn.Sequential(
            nn.Conv2d(64,64,3),
            nn.ReLU(),
            nn.MaxPool2d((2,2),stride = 1)
        )
        self.under_layer1 = nn.Sequential(
            nn.Conv2d(4,160,13),
            nn.ReLU()
        )
        self.final_layer = nn.Conv2d(224,5,21)
        #self.upper_conv1 = nn.Conv2d(4,64,7)
        #self.upper_conv2 = nn.Conv2d(64,64,3)
        #self.under_conv1 = nn.Conv2d(4,160,13)
        #self.final_conv = nn.Conv2d(224,5,21)
    
    def forward(self, x):
        upper_x = self.upper_layer2(self.upper_layer1(x))
        under_x = self.under_layer1(x)
        #upper_x = F.max_pool2d(F.relu(self.upper_conv1(x)), (4, 4),stride = 1)
        #upper_x = F.max_pool2d(F.relu(self.upper_conv2(upper_x)), (2, 2), stride = 1)
        #under_x = F.relu(self.under_conv1(x))
        final_x = torch.cat((under_x, upper_x), 1)
        out = F.softmax(self.final_layer(final_x))
        return out
        
net = TwoPathConv()
print(net)

x = Variable(torch.randn(1,4,33,33), requires_grad = True)
y_pred = net.forward(x)
#y_pred = y_pred.data.resize_(1,5)
#y_pred = Variable(y_pred,requires_grad = True)
#print(y_pred.size())
#print(y_pred.view)

TwoPathConv (
  (upper_layer1): Sequential (
    (0): Conv2d(4, 64, kernel_size=(7, 7), stride=(1, 1))
    (1): ReLU ()
    (2): MaxPool2d (size=(4, 4), stride=(1, 1), dilation=(1, 1))
  )
  (upper_layer2): Sequential (
    (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1))
    (1): ReLU ()
    (2): MaxPool2d (size=(2, 2), stride=(1, 1), dilation=(1, 1))
  )
  (under_layer1): Sequential (
    (0): Conv2d(4, 160, kernel_size=(13, 13), stride=(1, 1))
    (1): ReLU ()
  )
  (final_layer): Conv2d(224, 5, kernel_size=(21, 21), stride=(1, 1))
)


In [148]:
label = Variable(torch.LongTensor([3]), requires_grad=False)
learning_rate = 1e-4
optimizer = torch.optim.Adam(net.parameters(), lr=learning_rate)
for i in range(10):
    optimizer.zero_grad()
    y_pred = net.forward(x)
    y_pred = y_pred.resize(1,5)
    loss = F.nll_loss(F.log_softmax(y_pred), label)
    loss.backward()
    #print(loss)
    optimizer.step()
    

Variable containing:
 0.9048
[torch.FloatTensor of size 1]

Variable containing:
 0.9048
[torch.FloatTensor of size 1]

Variable containing:
 0.9048
[torch.FloatTensor of size 1]

Variable containing:
 0.9048
[torch.FloatTensor of size 1]

Variable containing:
 0.9048
[torch.FloatTensor of size 1]

Variable containing:
 0.9048
[torch.FloatTensor of size 1]

Variable containing:
 0.9048
[torch.FloatTensor of size 1]

Variable containing:
 0.9048
[torch.FloatTensor of size 1]

Variable containing:
 0.9048
[torch.FloatTensor of size 1]

Variable containing:
 0.9048
[torch.FloatTensor of size 1]



In [80]:
import torch
from torch.autograd import Variable

# N is batch size; D_in is input dimension;
# H is hidden dimension; D_out is output dimension.
N, D_in, H, D_out = 64, 1000, 100, 10

# Create random Tensors to hold inputs and outputs, and wrap them in Variables.
x = Variable(torch.randn(N, D_in))
y = Variable(torch.randn(N, D_out), requires_grad=False)

# Use the nn package to define our model and loss function.
model = torch.nn.Sequential(
    torch.nn.Linear(D_in, H),
    torch.nn.ReLU(),
    torch.nn.Linear(H, D_out),
)
loss_fn = torch.nn.MSELoss(size_average=False)

# Use the optim package to define an Optimizer that will update the weights of
# the model for us. Here we will use Adam; the optim package contains many other
# optimization algoriths. The first argument to the Adam constructor tells the
# optimizer which Variables it should update.
learning_rate = 1e-4
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
for t in range(500):
    # Forward pass: compute predicted y by passing x to the model.
    y_pred = model(x)

    # Compute and print loss.
    loss = loss_fn(y_pred, y)
    print(t, loss.data[0])

    # Before the backward pass, use the optimizer object to zero all of the
    # gradients for the variables it will update (which are the learnable weights
    # of the model)
    optimizer.zero_grad()

    # Backward pass: compute gradient of the loss with respect to model
    # parameters
    loss.backward()

    # Calling the step function on an Optimizer makes an update to its
    # parameters
    optimizer.step()

(0, 678.2203979492188)
(1, 660.6570434570312)
(2, 643.557373046875)
(3, 626.8915405273438)
(4, 610.68359375)
(5, 594.9915771484375)
(6, 579.7462158203125)
(7, 564.9285278320312)
(8, 550.4617919921875)
(9, 536.4539184570312)
(10, 522.885986328125)
(11, 509.7567443847656)
(12, 497.0480041503906)
(13, 484.7501525878906)
(14, 472.7842102050781)
(15, 461.11737060546875)
(16, 449.7706604003906)
(17, 438.8199157714844)
(18, 428.1475830078125)
(19, 417.7566223144531)
(20, 407.73785400390625)
(21, 397.9593505859375)
(22, 388.41912841796875)
(23, 379.1401062011719)
(24, 370.1069030761719)
(25, 361.2682189941406)
(26, 352.64422607421875)
(27, 344.20477294921875)
(28, 335.9613037109375)
(29, 327.9267883300781)
(30, 320.1400146484375)
(31, 312.5726013183594)
(32, 305.17242431640625)
(33, 297.9583435058594)
(34, 290.9154968261719)
(35, 284.0159606933594)
(36, 277.3037414550781)
(37, 270.73114013671875)
(38, 264.2970275878906)
(39, 258.0490417480469)
(40, 251.9435577392578)
(41, 245.95468139648438)
(

In [127]:

from graphviz import Digraph
import re
import torch
import torch.nn.functional as F
from torch.autograd import Variable
from torch.autograd import Variable
import torchvision.models as models


def make_dot(var):
    node_attr = dict(style='filled',
                     shape='box',
                     align='left',
                     fontsize='12',
                     ranksep='0.1',
                     height='0.2')
    dot = Digraph(node_attr=node_attr, graph_attr=dict(size="12,12"))
    seen = set()

    def add_nodes(var):
        if var not in seen:
            if isinstance(var, Variable):
                value = '('+(', ').join(['%d'% v for v in var.size()])+')'
                dot.node(str(id(var)), str(value), fillcolor='lightblue')
            else:
                dot.node(str(id(var)), str(type(var).__name__))
            seen.add(var)
            if hasattr(var, 'previous_functions'):
                for u in var.previous_functions:
                    dot.edge(str(id(u[0])), str(id(var)))
                    add_nodes(u[0])
    add_nodes(var.creator)
    return dot


inputs = torch.randn(1,3,224,224)
resnet18 = models.resnet18()
y = resnet18(Variable(inputs))
# print(y)
x = Variable(torch.randn(1,4,33,33), requires_grad = True)
y_pred = net.forward(x)

g = make_dot(y_pred)
g.view()
print(g)

ExecutableNotFound: failed to execute ['dot', '-Tpdf', '-O', 'Digraph.gv'], make sure the Graphviz executables are on your systems' PATH